## Might be usefull later, but for now useless

## Imports

In [31]:
from itertools import permutations

In [32]:
import random

## Vars

In [33]:
player_options = [
        ["X",None,None,
         None,None,None,
         None,None,None],

         [None,"X",None,
          None,None,None,
          None,None,None],

          [None,None,"X",
          None,None,None,
          None,None,None],

          [None,None,None,
          "X",None,None,
          None,None,None],

          [None,None,None,
          None,"X",None,
          None,None,None],

          [None,None,None,
          None,None,"X",
          None,None,None],

          [None,None,None,
          None,None,None,
          "X",None,None],

          [None,None,None,
          None,None,None,
          None,"X",None],

          [None,None,None,
          None,None,None,
          None,None,"X"]
]

In [34]:
class State:
  def __init__(self, env, value, parent, children):
    self.parent = parent
    self.children = children
    self.env = env
    self.value = value

In [35]:
# Root state
root_state = State([None,None,None,
                   None,None,None,
                   None,None,None], 
                   None,
                   None,
                   [])



In [50]:
curr_state = root_state

## Functions

In [37]:
# If exist in the tree, return it, if not, add it
def get_new_state(state, action):
  new_state = State([],0,state,[])

  if action == None:
    return state
  
  # Set the new state env after action
  for state_action in zip(state.env,action):
    if(state_action[0] == None and state_action[1] != None):
      new_state.env.append(state_action[1])
    else:
      new_state.env.append(state_action[0])

  # If this state env already in tree, return it
  if state.children != None:
    for child_state in state.children:
      if child_state.env == new_state.env:
        return child_state
  
  # If not, append this new state to tree and return it
  new_state.parent = state
  state.children.append(new_state)
  return new_state 

In [38]:
def player_turn(state, action):
  return get_new_state(state,action)

In [78]:
# Takes state and returns an action
def agent_turn(state):

  # Determine possible actions
  possible_actions = []
  for i in range(0,len(state.env)):
    if state.env[i] == None:
      new_action = [None] * 9
      new_action[i] = "O"
      possible_actions.append(new_action)

  # print("possible actions: ",possible_actions)
  # print("state: ", state.env)

  # Exploit or Expore
  # For now, we will always exploit, unless we have not seen this state
  # If we have not seen this state, we choose randomly, e.g. Explore
  exploit = True
  explore = False

  # Exploit: refer to seen states, choose action that yields highest value state
  if exploit:
    better_action_found = False
    best_action = possible_actions[0]
    best_value_state = get_new_state(state,best_action)

    for action in possible_actions[1:]: 
      new_possible_state = get_new_state(state,action)
      
      if new_possible_state.value > best_value_state.value:
        better_action_found = True
        best_action = action
      
      best_value_state = get_new_state(state,best_action)
  
  if explore or not better_action_found:
    # Explore: select from possible actions randomly
    best_action = random.choice(possible_actions)

  # Make the play
  return get_new_state(state,best_action)

In [40]:
def check_for_win(state):
  # Horizontal lines
  if state.env[0] == state.env[1] == state.env[2] != None:
    return state.env[1]
  if state.env[3] == state.env[4] == state.env[5] != None:
    return state.env[4]
  if state.env[6] == state.env[7] == state.env[8] != None:
    return state.env[7]
  # Vertical lines
  if state.env[0] == state.env[3] == state.env[6] != None:
    return state.env[3]
  if state.env[1] == state.env[4] == state.env[7] != None:
    return state.env[4]
  if state.env[2] == state.env[5] == state.env[8] != None:
    return state.env[5]
  # Diagonals
  if state.env[0] == state.env[4] == state.env[8] != None:
    return state.env[4]
  if state.env[2] == state.env[4] == state.env[6] != None:
    return state.env[4]

In [41]:
step_size = 0.1
def value_function(state):
  return step_size*(state.value - state.parent.value)

In [42]:
def back_propagate(state):
  state_iter = state
  while state_iter.parent.value != None:
    state_iter.parent.value = value_function(state_iter)
    state_iter = state_iter.parent

In [94]:
def print_state(state):
  print(str(state.env[0]).center(6,' '), " | ", str(state.env[1]).center(6,' '), " | ", str(state.env[2]).center(6,' '))
  print(str(state.env[3]).center(6,' '), " | ", str(state.env[4]).center(6,' '), " | ", str(state.env[5]).center(6,' '))
  print(str(state.env[6]).center(6,' '), " | ", str(state.env[7]).center(6,' '), " | ", str(state.env[8]).center(6,' '))

## Execution

In [ ]:
# Gameloop
curr_state = root_state
while (1):
  print("\nyour turn:")
  player_choice = input("\n0 | 1 | 2\n3 | 4 | 5\n6 | 7 | 8\n\n")

  curr_state = player_turn(curr_state,player_options[int(player_choice)])

  print_state(curr_state)

  if check_for_win(curr_state) == "X":
    curr_state.value = -1 # This was a loss for the agent
    back_propagate(curr_state)
    break
  elif None not in curr_state.env:
    curr_state = root_state
    break
    
  print("\nagent's turn...")
  curr_state = agent_turn(curr_state)

  print_state(curr_state)

  if check_for_win(curr_state) == "O":
    curr_state.value = 1 # This was a win for the agent
    back_propagate(curr_state)
    break
  elif None not in curr_state.env:
    curr_state = root_state
    break

In [82]:
# Randomly plays moves, for training!
curr_state = root_state
for i in range(0,1000000):

  player_choice = random.choice(player_options)
  curr_state = player_turn(curr_state, player_choice)

  if check_for_win(curr_state) == "X":
    curr_state.value = -1 # This was a loss for the agent
    back_propagate(curr_state)
    curr_state = root_state
    continue
  elif None not in curr_state.env:
    curr_state = root_state
    continue
  
  curr_state = agent_turn(curr_state)
  if check_for_win(curr_state) == "O":
    curr_state.value = 1 # This was a win for the agent
    back_propagate(curr_state)
    curr_state = root_state
    continue
  elif None not in curr_state.env:
    curr_state = root_state
    continue

## Traversing the state tree

In [ ]:
# Print most recently backpropagated path
state_iter = curr_state
while state_iter.value != None:
    print(state_iter.env)
    print(state_iter.value)
    state_iter = state_iter.parent

In [ ]:
# prints entire tree
# tree will be huge if 'trained', careful!
def traverse(state):
  print("\n")
  print(state.env, " : ", state.value)
  print("children:")
  for child in state.children:
    print(child.env, " : ", child.value)

  for child in state.children:
    traverse(child)

traverse(root_state)